In [4]:
import json, pickle, copy, os, time, random
from collections import Counter, defaultdict
import numpy as np
from pprint import pprint

In [6]:
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0818.json", "r"))
img_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0817_neg_ranked.json", "r"))

print(Counter([txt_dataset[k]['split'] for k in txt_dataset]))
print(len(set([txt_dataset[k]['Guid'] for k in txt_dataset])))

print(Counter([img_dataset[k]['split'] for k in img_dataset]))
print(Counter([img_dataset[k]['Qcate'] for k in img_dataset]))
print(len(set([img_dataset[k]['Guid'] for k in img_dataset])))

Counter({'train': 17812, 'test': 5084, 'val': 2455})
25351
Counter({'train': 16447, 'ood_test': 3987, 'val': 2511, 'ind_test': 2507})
Counter({'YesNo': 8429, 'Others': 6713, 'choose': 5240, 'number': 2339, 'color': 2069, 'shape': 662})
25452


In [2]:
tmp = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/TestSet5Answers_0818_from_Mridu.json", "r"))
print(type(tmp))

<class 'dict'>


In [10]:
a = set([img_dataset[k]['Guid'] for k in img_dataset if img_dataset[k]['split'] in ['ind_test', 'ood_test']])
a = a.union(set([txt_dataset[k]['Guid'] for k in txt_dataset if txt_dataset[k]['split'] in ['test']]))
print(set(list(tmp.keys())) - a)
print(a - set(list(tmp.keys())))

{'dc4dc3b8d46f11ebba07a504a20e7724', 'dc4f2460d46f11ebba07a504a20e7724'}
{'dc4c05a0d46f11ebba07a504a20e7724'}


In [29]:
### Need to modify the split a little bit. Some mismatch
move_to_test = ['dc4dc3b8d46f11ebba07a504a20e7724', 'dc4f2460d46f11ebba07a504a20e7724']
move_to_train = ['dc4c05a0d46f11ebba07a504a20e7724']
img_dataset_0818_add5x = copy.deepcopy(img_dataset)
txt_dataset_0818_add5x = copy.deepcopy(txt_dataset)

delete_img_count = 0
delete_txt_count = 0
total_count = 0
for k in img_dataset_0818_add5x:
    guid = img_dataset_0818_add5x[k]['Guid']
    if guid in move_to_test:
        print("img data k = {} move from {} to test".format(k, img_dataset_0818_add5x[k]['split']))
        img_dataset_0818_add5x[k]['split'] = 'ind_test'
    if guid in move_to_train:
        print("img data k = {} move from {} to train".format(k, img_dataset_0818_add5x[k]['split']))
        img_dataset_0818_add5x[k]['split'] = 'train'
    
    if not guid in tmp: continue
    total_count += 1
    
    annotation_5x = tmp[guid]
    assert annotation_5x['Question'] in img_dataset_0818_add5x[k]['Q']
    if annotation_5x['HasIssue']: 
        print("delete img data k = {}".format(k))
        del img_dataset_0818_add5x[k]
        delete_img_count += 1
        continue
    ori_A = img_dataset_0818_add5x[k]['A']
    img_dataset_0818_add5x[k]['A'] = [ori_A]
    for a in annotation_5x['Answers']:
        img_dataset_0818_add5x[k]['A'].append('"' + a + '"')

for k in txt_dataset_0818_add5x:
    guid = txt_dataset_0818_add5x[k]['Guid']
    if guid in move_to_test:
        print("txt data k = {} move from {} to test".format(k, txt_dataset_0818_add5x[k]['split']))
        txt_dataset_0818_add5x[k]['split'] = 'test'
    if guid in move_to_train:
        print("txt data k = {} move from {} to train".format(k, txt_dataset_0818_add5x[k]['split']))
        txt_dataset_0818_add5x[k]['split'] = 'train'
    
    if not guid in tmp: continue
    total_count += 1
    
    annotation_5x = tmp[guid]
    assert annotation_5x['Question'] in txt_dataset_0818_add5x[k]['Q']
    if annotation_5x['HasIssue']: 
        print("delete txt data k = {}".format(k))
        del txt_dataset_0818_add5x[k]
        delete_txt_count += 1
        continue
    ori_A = txt_dataset_0818_add5x[k]['A']
    txt_dataset_0818_add5x[k]['A'] = [ori_A]
    for a in annotation_5x['Answers']:
        txt_dataset_0818_add5x[k]['A'].append('"' + a + '"')

print("total_count = ", total_count)
print("delete_img_count = ", delete_img_count)
print("delete_txt_count = ", delete_txt_count)
print(len(img_dataset_0818_add5x))
print(len(txt_dataset_0818_add5x))

TypeError: string indices must be integers

In [ ]:
json.dump(img_dataset_0818_add5x, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0818_add5x.json", "w"), indent=4)
json.dump(txt_dataset_0818_add5x, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0818_add5x.json", "w"), indent=4)

In [25]:
while True:
    k = random.choice(list(img_dataset.keys()))
    if img_dataset[k]['split'] in ['ind_test', 'ood_test']:
        print(k)
        break

17535


In [26]:
print(img_dataset['17535']['Guid'])
print(img_dataset['17535']['Q'])
print(img_dataset['17535']['A'])

dc6b66c0d46f11ebba07a504a20e7724
"Do the Parade room of the Hotel de Chevreuse in the Louvre Museum and the Turkish Cabinet room in the Louvre Museum both employ tapestries as decoration?"
"No, only the Parade room of the Hotel de Chevreuse uses tapestries."


In [27]:
tmp['dc6b66c0d46f11ebba07a504a20e7724']

'Question: "Do the Parade room of the Hotel de Chevreuse in the Louvre Museum and the Turkish Cabinet room in the Louvre Museum both employ tapestries as decoration?", Answers: [\'Yes, the Parade room of the Hotel de Chevreuse in the Louvre Museum and the Turkish Cabinet room in the Louvre Museum both employ tapestries as decoration.\', \'The Parade room of the Hotel de Chevreuse in the Louvre Museum and the Turkish Cabinet room in the Louvre Museum do not both employ tapestries as decoration.\', \'No, although the Parade room of the Hotel de Chevreuse in the Louvre Museum employs tapestries as decoration, the Turkish Cabinet room in the Louvre Museum does not.\', \'The Parade room of the Hotel de Chevreuse in the Louvre Museum and the Turkish Cabinet room in the Louvre Museum both employ tapestries as decoration.\', \'The Parade room of the Hotel de Chevreuse in the Louvre Museum and the Turkish Cabinet room in the Louvre Museum do both employ tapestries as decoration.\'], HasIssue: F

In [34]:
for k in tmp:
    print(k)
    print(tmp[k])
    break

dc62fd0ad46f11ebba07a504a20e7724
Question: "Are all the feathers on the Dark-rumped Rosefinch of the same general color?", Answers: ['All of the feathers on the Dark-rumped Rosefinch have the same generally reddish/violet color.', 'Yes, all the feathers on the Dark-rumped Rosefinch are the same color.', 'Yes, all of the feathers on the Dark-rumped Rosefinch are the same color.', 'Yes, all of the feathers on the Dark-rumped Rosefinch are of the same general color.', 'Yes, all the feathers on the Dark-rumped Rosefinch are of the same general color.'], HasIssue: False


In [35]:
type(json.loads('{'+tmp['dc62fd0ad46f11ebba07a504a20e7724']+'}'))

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)